<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/ttest-greta/dataset_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()

interactive(children=(Text(value='', description='Email', placeholder='Enter email'), Text(value='', descripti…

In [14]:
import dataset
import importlib
import raster
importlib.reload(dataset)

<module 'dataset' from '/content/gdrive/MyDrive/ttest-greta/ddf_common/dataset.py'>

# preprocess_sample_data Tests

In [18]:
from pandas.testing import assert_frame_equal

def test_partition_data_fixed():
  '''
  For reference:
  _FIXED_PARTITION_STRATEGY = FixedPartitionStrategy(
    # Train
    DatasetGeographicPartitions(
        min_longitude=-62.5,
        max_longitude=float('inf'),
        min_latitude=-5,
        max_latitude=float('inf'),
    ),
    # Validation
    DatasetGeographicPartitions(
        min_longitude=float('-inf'),
        max_longitude=-62.5,
        min_latitude=-5,
        max_latitude=float('inf')
    ),
    # Test
    DatasetGeographicPartitions(
        min_longitude=float('-inf'),
        max_longitude=float('inf'),
        min_latitude=float('-inf'),
        max_latitude=-5
    )
  )
  '''
  train_long_values = [-62.5, -50.0, -20.0, 0.0, 100.0]
  train_lat_values = [-5.0, -3.0, -2.0, -1.0, 0.0]

  validation_long_values = [-80.0, -70.0, -63.0]
  validation_lat_values = [-5.0, -2.0, 5.0]

  test_long_values = [-50.0, -30.0, -40.0, 0.0, 50.0, 100.0]
  test_lat_values = [-50.0, -30.0, -40.0, -20.0, -10.0, -5.1]

  test_data = pd.DataFrame({
      "long": train_long_values + test_long_values + validation_long_values,
      "lat": train_lat_values + test_lat_values + validation_lat_values,
  })

  partitioned_dataset = dataset.partition(test_data, dataset.PartitionStrategy.FIXED)

  assert_frame_equal(
      partitioned_dataset.train.reset_index(drop=True),
      pd.DataFrame({
          "long": train_long_values,
          "lat": train_lat_values,
      }).reset_index(drop=True)
  )
  assert_frame_equal(
      partitioned_dataset.validation.reset_index(drop=True),
      pd.DataFrame({
          "long": validation_long_values,
          "lat": validation_lat_values,
      }).reset_index(drop=True)
  )
  assert_frame_equal(
      partitioned_dataset.test.reset_index(drop=True),
      pd.DataFrame({
          "long": test_long_values,
          "lat": test_lat_values,
      }).reset_index(drop=True)
  )

test_partition_data_fixed()

# create_fraudulent_samples Tests

In [19]:
TEST_ISOSCAPE_MEANS_FILENAME = "xgb_means_oxygen_isoscape_ucd_42.tiff" #@param
TEST_ISOSCAPE_VARS_FILENAME = "xgb_variances_oxygen_isoscape_ucd_42.tiff" #@param

In [20]:
def create_fraudulent_samples_test():
  # The range of expected values is around 25 for most locations here.
  real_samples_data = pd.DataFrame({
      'Code': ["a", "a", "a", "a", "a", "b", "b", "b","b","b"],
      'lat': [-2.499,-2.499,-2.499,-2.499,-2.499, -6.009706576,-6.009706576,-6.009706576,-6.009706576,-6.009706576],
      'long': [-59.121, -59.121, -59.121, -59.121, -59.121, -61.8686565,-61.8686565,-61.8686565,-61.8686565,-61.8686565],
      'd18O_cel': [-20.0, 5.0, 100.0, 5.0, 8.0, 5.0, 0.0, 3.0,
                   -20.0, 25.0],
  })

  mean_iso = raster.load_raster(raster.get_raster_path(TEST_ISOSCAPE_MEANS_FILENAME), use_only_band_index=0)
  element = 'd18O_cel'
  max_trusted_radius = 0.1
  max_fraud_radius = 30
  min_fraud_radius = 5

  fake_sample = (
    dataset.create_fraudulent_samples(real_samples_data,
                                      mean_iso,
                                      element,
                                      max_trusted_radius,
                                      max_fraud_radius,
                                      min_fraud_radius))

create_fraudulent_samples_test()

Driver: GTiff/GeoTIFF
Size is 940 x 936 x 1
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-73.975139313, 5.266527396)
Pixel Size = (0.0416666665, -0.041666666500000005)
